In [2]:
from importlib.metadata import version

In [ ]:
pip install protobuf sentencepiece

In [ ]:
pip install --upgrade transformers

In [10]:
version('transformers')

'4.45.2'

In [ ]:
pip install --upgrade diffusers

In [3]:
version('diffusers')

'0.30.3'

In [ ]:
pip install --upgrade optimum-quanto

In [7]:
version('optimum-quanto')

'0.2.5'

https://huggingface.co/black-forest-labs/FLUX.1-schnell

https://huggingface.co/docs/diffusers/main/en/api/pipelines/flux

https://gist.github.com/sayakpaul/b664605caf0aa3bf8585ab109dd5ac9c

https://gist.github.com/sayakpaul/e1f28e86d0756d587c0b898c73822c47

https://github.com/huggingface/diffusers/issues/9165

https://github.com/lllyasviel/stable-diffusion-webui-forge/discussions/981

https://huggingface.co/silveroxides/flux1-nf4-weights

In [1]:
import time
import torch
from optimum.quanto import freeze, qfloat8, qint4, quantize
from diffusers import FlowMatchEulerDiscreteScheduler, AutoencoderKL
from diffusers.models.transformers.transformer_flux import FluxTransformer2DModel
from diffusers.pipelines.flux.pipeline_flux import FluxPipeline
from transformers import CLIPTextModel, CLIPTokenizer,T5EncoderModel, T5TokenizerFast

# Load the model
scheduler = FlowMatchEulerDiscreteScheduler.from_pretrained("black-forest-labs/FLUX.1-schnell", subfolder="scheduler", revision="refs/pr/1")

In [2]:
# Model download: 1.71 GB
# VRAM before: 180 MB
# VRAM after: 610 MB

text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.bfloat16).to("cuda")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.bfloat16)

In [3]:
# Model download: 9.52 GB
# VRAM before: 610 MB
# VRAM after: 

text_encoder_2 = T5EncoderModel.from_pretrained("black-forest-labs/FLUX.1-schnell", subfolder="text_encoder_2", torch_dtype=torch.bfloat16, revision="refs/pr/1")
tokenizer_2 = T5TokenizerFast.from_pretrained("black-forest-labs/FLUX.1-schnell", subfolder="tokenizer_2", torch_dtype=torch.bfloat16, revision="refs/pr/1")

# quantize to 8-bit to fit on an RTX 4090
# quantize(text_encoder_2, weights=qfloat8)
# quantize to 4-bit to fit on an RTX 3070
quantize(text_encoder_2, weights=qint4)
freeze(text_encoder_2)
text_encoder_2.to("cuda")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/root/miniforge3/envs/pytorch-2.4/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1965: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


T5EncoderModel(
  (shared): Embedding(32128, 4096)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 4096)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): QLinear(in_features=4096, out_features=4096, bias=False)
              (k): QLinear(in_features=4096, out_features=4096, bias=False)
              (v): QLinear(in_features=4096, out_features=4096, bias=False)
              (o): QLinear(in_features=4096, out_features=4096, bias=False)
              (relative_attention_bias): Embedding(32, 64)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): QLinear(in_features=4096, out_features=10240, bias=False)
              (wi_1): QLinear(in_features=4096, out_features=10240, bias=False)
           

In [ ]:
# Model download: GB
# VRAM before: 
# VRAM after: 

vae = AutoencoderKL.from_pretrained("black-forest-labs/FLUX.1-schnell", subfolder="vae", torch_dtype=torch.bfloat16, revision="refs/pr/1").to("cuda")

In [ ]:
# Model download: GB
# VRAM before: 
# VRAM after: 

transformer = FluxTransformer2DModel.from_pretrained("black-forest-labs/FLUX.1-schnell", subfolder="transformer", torch_dtype=torch.bfloat16, revision="refs/pr/1")
# quantize to 8-bit to fit on an RTX 4090
#quantize(transformer, weights=qfloat8)
# quantize to 4-bit to fit on an RTX 3070
quantize(transformer, weights=qint4)
freeze(transformer)
transformer.to("cuda")

pipe = FluxPipeline(
    scheduler=scheduler,
    text_encoder=text_encoder,
    tokenizer=tokenizer,
    text_encoder_2=None,
    tokenizer_2=tokenizer_2,
    vae=vae,
    transformer=None,
)
pipe.text_encoder_2 = text_encoder_2
pipe.transformer = transformer
pipe.enable_model_cpu_offload()

def generate_image(prompt):
    # Generate image from prompt
    image = pipe(
        prompt=prompt, 
        width=1024,
        height=1024,
        num_inference_steps=4, 
        generator=torch.Generator().manual_seed(int(time.time())),
        guidance_scale=3.5,
    ).images[0]
    
    return image

In [ ]:
generate_image("un lapin dort dans son un panier")